In [4]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

# 1. Создаем маленький датасет
data = {
    'text': [
        'Фильм был отличный, очень понравился!',
        'Ужасный опыт, не рекомендую.',
        'Нормально, но могло быть лучше.',
        'Лучший сервис, буду обращаться еще!',
        'Разочарован, ожидал большего.'
    ],
    'label': [1, 0, 0, 1, 0]
}

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# 2. Загружаем модель
model_name = "blanchefort/rubert-base-cased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


# 3. Токенизируем
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 4. Параметры тренировки
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10,
    logging_steps=5
)

# 5. Тренер и тренировка
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

trainer.train()

# 6. Проверка
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Модель на правильное устройство

test_texts = [
    "Очень доволен покупкой!",
    "Качество ужасное, деньги на ветер."
]

for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64)

    # Все данные на то же устройство, что и модель
    inputs = {key: value.to(device) for key, value in inputs.items()}

    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1)
    sentiment = "ПОЗИТИВ" if prediction == 1 else "НЕГАТИВ"
    print(f"Текст: {text}")
    print(f"Результат: {sentiment}\n")

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: blanchefort/rubert-base-cased-sentiment
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=1.1.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=1.1.0'`

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Объяснение: GPT - это декодерная модель. Генерирует текст последовательно, слово за словом.
# Хороша для генерации текста: чат-боты, продолжение текста, творчество.

# 1. Загружаем модель и токенизатор
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Устанавливаем padding token
tokenizer.pad_token = tokenizer.eos_token


# 2. Функция для генерации текста с разными параметрами
def generate_text(prompt, temperature=1.0, top_k=50, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        inputs.input_ids,
        max_length=50,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


# 3. Промпт для генерации
prompt = "Сегодня был прекрасный день"

print("=== Генерация текста с разными параметрами ===\n")

# Разные temperature
print("1. Разная temperature (влияет на случайность):")
print(f"   temperature=0.7 (более предсказуемо):")
print(f"   {generate_text(prompt, temperature=0.7, top_k=50, top_p=0.9)[:100]}...")
print(f"\n   temperature=1.0 (стандарт):")
print(f"   {generate_text(prompt, temperature=1.0, top_k=50, top_p=0.9)[:100]}...")
print(f"\n   temperature=1.5 (более креативно):")
print(f"   {generate_text(prompt, temperature=1.5, top_k=50, top_p=0.9)[:100]}...")

# Разные top_k
print("\n\n2. Разный top_k (сколько вариантов рассматривать):")
print(f"   top_k=50 (только 50 лучших вариантов):")
print(f"   {generate_text(prompt, temperature=1.0, top_k=50, top_p=0.9)[:100]}...")
print(f"\n   top_k=100 (100 лучших вариантов):")
print(f"   {generate_text(prompt, temperature=1.0, top_k=100, top_p=0.9)[:100]}...")

# Разные top_p
print("\n\n3. Разный top_p (nucleus sampling):")
print(f"   top_p=0.9 (рассматриваем варианты с суммарной вероятностью 0.9):")
print(f"   {generate_text(prompt, temperature=1.0, top_k=50, top_p=0.9)[:100]}...")
print(f"\n   top_p=0.95 (рассматриваем больше вариантов):")
print(f"   {generate_text(prompt, temperature=1.0, top_k=50, top_p=0.95)[:100]}...")

# Объяснение параметров:
# - temperature: чем выше, тем более случайный текст
# - top_k: ограничивает выбор только K лучшими вариантами
# - top_p: выбирает из минимального набора вариантов, чья сумма вероятностей ≥ p